# Base de données: interroger une base de données MySQL

## Imports

In [1]:
import os # exécuter du bash dans l'os?
import sqlite3 # exécuter du sql avec python grâce à sqlite3
import zipfile # pour dézipper
from pathlib import Path # se promener dans l'arborescence des fichiers
import requests # pour adresser sur le web

# Créer un dossier data avec 'mkdir' si le chemin n'existe pas encore

if not os.path.exists("../data"):
    os.mkdir("../data")

    

## Créer une base de données SQLite

In [2]:
# Créer un répertoire DB s'il n'existe pas encore

db_path = '../data/db'
nom_variable = '../data/essai'

if not os.path.exists(nom_variable):
    os.mkdir(nom_variable)

In [5]:
# Récupérer le fichier ZIP qui contient la DB de test

url = "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip"

filename = url.split("/")[-1] # préparer le nom du fichier à partir de l'url en parcourant à l'envers et en prenant le dernier élément après avoir spliter l'url à l'aide du "/"
filename2 = url.split("/")[-2]

print(filename)
print(filename2)

# Récupérer le fichier zip dans la RAM

response = requests.get(url) # adressage avec 'get'

chinook.zip
03


In [6]:
# Ecrire le fichier sur le disque avec la méthode 'open' qui joint le nom du fichier à l'endroit d'écriture

with open(os.path.join(db_path, filename), 'wb') as f: # fonction définie comme 'f' pour écrire le contenu de 'response' qui chope le contenu à l'endroit de l'url
    f.write(response.content)

In [7]:
# help(open) # 'w' pour écrire le contenu, 'b' en mode binaire

In [8]:
# Extraire le zip pour obtenir la db

with zipfile.ZipFile(os.path.join(db_path, filename), 'r') as zip_ref: # lire le contenu du fichier zip et extraire le contenu dans le dossier 'db' et lui donner le nom du fichier voulu
    zip_ref.extractall(db_path)

## Schéma de la base de données

<img src="https://cdn.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg" />

## Se connecter à la base de données

In [9]:
conn = sqlite3.connect(os.path.join(db_path, "chinook.db")) # stocker la requête de connexion dans une variable
c = conn.cursor() # nommer le curseur pour exécuter les requêtes sur la db

## Trouver l'identifiant de Miles Davis

In [21]:
artist = 'Miles Davis'

query1 = f"""
    SELECT
        artistId
    FROM
        artists
    WHERE
        name = '{artist}';
"""
# écrire la requête dans un f-string 

ad = c.execute(query1)
artist_id = c.fetchone()[0]

print(ad)
print(artist_id)

68


In [22]:
help(c.fetchone())

Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



## Trouver le nom du premier artiste de la DB

In [38]:
query1b = f"""
    SELECT
        name
    FROM
        artists
    WHERE
        artistId = 1;
"""
c.execute(query1b)
name = c.fetchone()[0]

print(name)

AC/DC


In [59]:
query1c = f"""
    SELECT
        name
    FROM
        artists;
"""
name_artists = set()

for row in c.execute(query1c):
    name_artists.add(row[0])

print(len(name_artists))
print(name_artists)

275
{'Pearl Jam', 'Habib Koité and Bamada', 'Audioslave', 'UB40', 'João Gilberto', "Christopher O'Riley", 'C. Monteverdi, Nigel Rogers - Chiaroscuro; London Baroque; London Cornett & Sackbu', 'Accept', 'Led Zeppelin', 'Jota Quest', 'Battlestar Galactica', 'Santana Feat. Rob Thomas', 'Julian Bream', 'Chico Buarque', 'Gerald Moore', 'Stereo Maracana', 'Adrian Leaper & Doreen de Feis', 'Elis Regina', 'Eugene Ormandy', 'Anne-Sophie Mutter, Herbert Von Karajan & Wiener Philharmoniker', 'The 12 Cellists of The Berlin Philharmonic', 'Apocalyptica', 'Motörhead', 'Toquinho & Vinícius', 'Chicago Symphony Orchestra & Fritz Reiner', 'Antal Doráti & London Symphony Orchestra', 'Vinícius E Qurteto Em Cy', 'Soundgarden', 'Academy of St. Martin in the Fields & Sir Neville Marriner', 'Velvet Revolver', 'Felix Schmidt, London Symphony Orchestra & Rafael Frühbeck de Burgos', 'James Brown', 'Raimundos', 'Deep Purple', 'Temple of the Dog', 'Fernanda Porto', 'Marvin Gaye', 'Legião Urbana', 'Alberto Turco & 

In [75]:
query1d = f"""
    SELECT
        artistId
    FROM
        artists;
"""
id_artists = set()

for row in c.execute(query1d):
    id_artists.add(str(row[0]))

print(len(id_artists))
print(id_artists)

275
{'263', '107', '156', '15', '242', '123', '201', '79', '38', '192', '147', '105', '189', '208', '120', '257', '179', '53', '91', '127', '237', '268', '164', '104', '218', '231', '83', '185', '176', '74', '219', '170', '5', '187', '230', '117', '6', '70', '94', '89', '14', '63', '161', '77', '182', '65', '146', '221', '28', '52', '275', '131', '149', '78', '246', '239', '23', '87', '145', '260', '165', '226', '11', '143', '175', '169', '140', '110', '160', '92', '114', '236', '122', '266', '224', '125', '10', '150', '212', '233', '210', '56', '171', '228', '61', '42', '43', '72', '136', '157', '85', '139', '158', '255', '50', '199', '188', '130', '58', '247', '44', '223', '46', '21', '234', '113', '244', '57', '135', '41', '88', '121', '207', '250', '106', '29', '132', '90', '206', '99', '193', '64', '39', '51', '126', '220', '248', '36', '138', '166', '184', '240', '108', '33', '245', '17', '97', '134', '103', '34', '35', '95', '45', '30', '191', '37', '102', '151', '200', '225', '

In [48]:
help(conn.cursor())

Help on Cursor object:

class Cursor(builtins.object)
 |  SQLite database cursor class.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  close(...)
 |      Closes the cursor.
 |  
 |  execute(...)
 |      Executes a SQL statement.
 |  
 |  executemany(...)
 |      Repeatedly executes a SQL statement.
 |  
 |  executescript(...)
 |      Executes a multiple SQL statements at once. Non-standard.
 |  
 |  fetchall(...)
 |      Fetches all rows from the resultset.
 |  
 |  fetchmany(...)
 |      Fetches several rows from the resultset.
 |  
 |  fetchone(...)
 |      Fetches one row from the resultset.
 |  
 |  setinputsizes(...)
 |      Required by DB-API. Does nothing in pysqlite.
 |  
 |  setoutputsize(...)
 |      Required by DB-API. Does nothing in pysqlite.
 |  
 |  ---

## Trouver les identifiants et les titres d'albums de Miles Davis

In [39]:
query2 = f"""
    SELECT
        albumId, title
    FROM 
        albums 
    WHERE 
        artistId = '{artist_id}';
"""
albums_ids = []
title = []

for row in c.execute(query2):
    albums_ids.append(str(row[0]))
    title.append(row[1]) # sélectionner la 2e donnée de la matrice générée par la requête

print(albums_ids)
print(title)

['48', '49', '157']
['The Essential Miles Davis [Disc 1]', 'The Essential Miles Davis [Disc 2]', 'Miles Ahead']


## Trouver les morceaux des albums de Miles Davis

In [40]:
query3 = f"""
    SELECT
        name,
        albumId
    FROM
        tracks
    WHERE 
        albumId IN ({",".join(albums_ids)});
"""

songs = set()
for row in c.execute(query3):
    songs.add(row[0])

print(f"\n{len(songs)} distinct songs found:\n")
for song in sorted(songs):
    print(song)


36 distinct songs found:

'Round Midnight
Black Satin
Blues For Pablo
Blues For Pablo (Alternate Take)
Bye Bye Blackbird
Compulsion
E.S.P.
Generique
I Don't Wanna Be Kissed (By Anyone But You)
I Don't Wanna Be Kissed (By Anyone But You) (Alternate Take)
Jean Pierre (Live)
Jeru
Lament
Little Church (Live)
Miles Ahead
Miles Runs The Voodoo Down
My Funny Valentine (Live)
My Ship
Nefertiti
New Rhumba
Now's The Time
Petits Machins (Little Stuff)
Portia
So What
Someday My Prince Will Come
Springsville
Springsville (Alternate Take)
Summertime
Tempus Fugit
The Duke
The Maids Of Cadiz
The Meaning Of The Blues
The Meaning Of The Blues/Lament (Alternate Take)
The Pan Piper
Time After Time
Walkin'


In [79]:
query3d = f"""
    SELECT
        title
    FROM
        albums
    WHERE 
        artistId IN ({",".join(id_artists)});
"""

albums = set()
for row in c.execute(query3d):
    albums.add(row[0])

print(f"\n{len(albums)} distinct albums found:\n")
for album in sorted(albums):
    print(album)


347 distinct albums found:

...And Justice For All
20th Century Masters - The Millennium Collection: The Best of Scorpions
A Copland Celebration, Vol. I
A Matter of Life and Death
A Real Dead One
A Real Live One
A Soprano Inspired
A TempestadeTempestade Ou O Livro Dos Dias
A-Sides
Ace Of Spades
Achtung Baby
Acústico
Acústico MTV
Acústico MTV [Live]
Adams, John: The Chairman Dances
Adorate Deum: Gregorian Chant from the Proper of the Mass
Afrociberdelia
Album Of The Year
Alcohol Fueled Brewtality Live! [Disc 1]
Alcohol Fueled Brewtality Live! [Disc 2]
All That You Can't Leave Behind
Allegri: Miserere
American Idiot
Angel Dust
Ao Vivo [IMPORT]
Appetite for Destruction
Aquaman
Are You Experienced?
Armada: Music from the Courts of England and Spain
Arquivo II
Arquivo Os Paralamas Do Sucesso
As Canções de Eu Tu Eles
Audioslave
Axé Bahia 2001
B-Sides 1980-1990
BBC Sessions [Disc 1] [Live]
BBC Sessions [Disc 2] [Live]
Bach: Goldberg Variations
Bach: Orchestral Suites Nos. 1 - 4
Bach: The Bra

## En une seule requête...

In [10]:
query4 = f"""
    SELECT
        tracks.name,
        tracks.albumId
    FROM
        artists
        JOIN albums ON artists.artistId = albums.artistId
        JOIN tracks ON albums.albumId = tracks.albumId
    WHERE 
        artists.name = '{artist}';
"""

songs = set()
for row in c.execute(query4):
    songs.add(row[0])

print(f"\n{len(songs)} distinct songs found:\n")
for song in sorted(songs):
    print(song)


36 distinct songs found:

'Round Midnight
Black Satin
Blues For Pablo
Blues For Pablo (Alternate Take)
Bye Bye Blackbird
Compulsion
E.S.P.
Generique
I Don't Wanna Be Kissed (By Anyone But You)
I Don't Wanna Be Kissed (By Anyone But You) (Alternate Take)
Jean Pierre (Live)
Jeru
Lament
Little Church (Live)
Miles Ahead
Miles Runs The Voodoo Down
My Funny Valentine (Live)
My Ship
Nefertiti
New Rhumba
Now's The Time
Petits Machins (Little Stuff)
Portia
So What
Someday My Prince Will Come
Springsville
Springsville (Alternate Take)
Summertime
Tempus Fugit
The Duke
The Maids Of Cadiz
The Meaning Of The Blues
The Meaning Of The Blues/Lament (Alternate Take)
The Pan Piper
Time After Time
Walkin'


## Quels sont les genres associés aux morceaux de Queen

In [90]:
query5 = f"""
    SELECT
        genres.name
    FROM
        genres
        JOIN tracks ON genres.genreId = tracks.genreId
        JOIN albums ON albums.albumId = tracks.albumId
        JOIN artists ON artists.artistId = albums.artistId
    WHERE 
        artists.name = '{'Queen'}';
"""

genres = set()
for row in c.execute(query5):
    genres.add(row[0])

print(f"\n{len(genres)} distinct genres found:\n")
for genre in sorted(genres):
    print(genre)


1 distinct genres found:

Rock


## Pour en savoir plus

- Tutoriel SQLite : https://www.sqlitetutorial.net/